Now,we have both uncleaned and cleaned dataset's .let's see how a fine tuned hate speech model performs in both the scenarios.

### Importing the libraries

In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline#for model in huggingface
from sklearn.metrics import accuracy_score, f1_score #my evaluation metrics

### Let's start with the uncleaned data

In [ ]:
# Loading the dataset
df = pd.read_excel("final_tweets.xlsx")
df.head()

,Text,Annotator-1,Annotator-2,gold labels
0,You don't need a special occasion to tell some...,0,0,0
1,Thanks for your insight Colleen.Radiation is c...,0,0,0
2,End each day by journaling 3 things you’re gr...,0,0,0
3,First this shit and now jaguar pulling som bul...,1,1,1
4,And fuck you to the guy I comforted when anoth...,1,1,1


In [ ]:
# 'Text' contains the tweets and 'Annotator-1' contains the main labels(my labels),i wil convert them into list for easy processing
texts = df['Text'].tolist()
true_labels = df['gold labels'].tolist()

### Loading the model
https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment
and refered from the model documentation

In [ ]:
# Load Twitter-roBERTa-base model and tokenizer
model_name = "cardiffnlp/twitter-roberta-base-sentiment"  # Pre-trained for sentiment and toxicity
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
# Define the classification pipeline(AS in model documentation)
classifier = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=True)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


### Model training

Here, I follow the **zero-shot** approach. I didn't fine tuned the model and will directly feed my uncleaned dataset to the model for the training and let's see how it performs.



The model has pre-trained labels as LABEL_1(NEUTRAL),LABEL_0(NEGATIVE/HATE/OFFENSIVE) as the model is pre-trained and fine-tuned based on those labels;such that model understands my case based on those . From the official documentation of the model, i can clearly state that my labels
 mostly matches the model's labels as they are written on the similar pattern of annotation guidelines.

we extract  hate and non_hate scores for each row from the intial model and then when hate_score>non_hate_score,then it is labelled as'1';else'0' and the predicted labels are stored in the dataframe

In [ ]:
# Performming th classification
#intialise the empty list to store the predictions
predictions = []
#starting a for loop in order to iterate each sample
for text in df['Text']:
  #using the classifier (model pipeline) to predict scores for the text
    result = classifier(text)

    # Access the first element of the outer list given by the pipeline.Since the outer list is quite unnecessary when processing a single input, result[0] extracts the inner list containing the predictions.
    result = result[0]  # This gives you the inner list;without this ,it gave me errors for the label extraction process.

    try:
        # Extract scores for 'LABEL_1' (neutral) and 'LABEL_0' (negative/hate) from the model's output;it results a dictionary {hate_score,non_hate_score}
        non_hate_score = next(item['score'] for item in result if item['label'] == "LABEL_1")  # Neutral sentiment
        hate_score = next(item['score'] for item in result if item['label'] == "LABEL_0")  # Negative/hate sentiment of the model
        #if a key error occurs, i have got this argument to ignore that , this generally happened to a glitch. i searched for this in stackoverflow,it suggested me this.
    except KeyError as e:
        print(f"Unexpected output structure: {result}")
        raise e

#now, it gave scores like for example for a row ;hate_score:0.78,non_hate 0.22, so i have assigned a condition in which if hate score>non hate, it will label as '1',otherwise '0'which matches my labelling scenario.

    # Assign the predicted label (1 for HATE, 0 for NON_HATE)
    pred_label = 1 if hate_score > non_hate_score else 0
    predictions.append(pred_label)

# Add predictions to the dataframe
df['predicted_label'] = predictions



In [ ]:
#Calculate evaluation metrics
# calculate accuracy
accuracy = accuracy_score(true_labels, df['predicted_label'])

# calculate F1 score (binary classification, average='binary' by default)
f1 = f1_score(true_labels, df['predicted_label'])

# Print the results
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.92
F1 Score: 0.9191919191919192


In [ ]:
df.head(15)

,Text,Annotator-1,Annotator-2,gold labels,predicted_label
0,You don't need a special occasion to tell some...,0,0,0,0
1,Thanks for your insight Colleen.Radiation is c...,0,0,0,0
2,End each day by journaling 3 things you’re gr...,0,0,0,0
3,First this shit and now jaguar pulling som bul...,1,1,1,1
4,And fuck you to the guy I comforted when anoth...,1,1,1,1
5,heir character design is horrendous. SweetBaby...,1,0,1,1
6,With the right people there is hope for change...,0,0,0,0
7,Even if KD was playing The Suns are just so sm...,1,1,1,1
8,Trump was elected a mere couple weeks ago and ...,1,1,1,1
9,"""Believe in your ability to create a better fu...",0,0,0,0


In [ ]:
#saving the predicted labels to the dataset
df.to_excel("final_uncleaned_predicted__data.xlsx", index=False)

### Let's look at the  cleaned data and perform zero-shot on that
(code is same for the below part,i just changed the dataset)
---



In [44]:
# Loading the dataset
data = pd.read_excel("final_tweets_cleaned .xlsx")
ctexts = data['Text'].tolist()
ctrue_labels = data['gold labels'].tolist()

### Model Training
Directly feeding the cleaned dataset to the model in the same zero-shot way

In [45]:

#intialise the empty list to store the predictions
predictions = []
#starting a for loop in order to iterate each sample
for text in data['Text']:
  #using the classifier (model pipeline) to predict scores for the text
    result = classifier(text)
# Access the first element of the outer list given by the pipeline.Since the outer list is quite unnecessary when processing a single input, result[0] extracts the inner list containing the predictions.
    result = result[0]  # This gives you the inner list;without this ,it gave me errors for the label extraction process.

    try:
        # Extract scores for 'LABEL_1' (neutral) and 'LABEL_0' (negative/hate) from the model's output,it results a dictionary {hate_score,non_hate_score}
        non_hate_score = next(item['score'] for item in result if item['label'] == "LABEL_1")  # Neutral sentiment
        hate_score = next(item['score'] for item in result if item['label'] == "LABEL_0")  # Negative/hate sentiment of the model
  #if a key error occurs, i have got this argument to ignore that , this generally happened to a glitch. i searched for this in stackoverflow,it suggested me this.
    except KeyError as e:
        print(f"Unexpected output structure: {result}")
        raise e

    #now, it gave scores like for example hate_score:0.78,non_hate 0.22, so i have assigned a condition in which if hate score>non hate, it will label as '1',otherwise '0'which matches my labelling scenario.
    # Assign the predicted label (1 for HATE, 0 for NON_HATE)
    pred_label = 1 if hate_score > non_hate_score else 0
    predictions.append(pred_label)

# Add predictions to the dataframe
data['predicted_label'] = predictions


In [46]:
accuracy = accuracy_score(ctrue_labels, data['predicted_label'])

# Calculate F1 score (binary classification, average='binary' by default)
f1 = f1_score(ctrue_labels, data['predicted_label'])

# Print the results
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.925
F1 Score: 0.9238578680203046


In [47]:
data.head(15)

,Text,Annotator-1,Annotator-2,gold labels,predicted_label
0,you dont need a special occasion to tell someo...,0,0,0,0
1,thanks for your insight colleenradiation is ce...,0,0,0,0
2,end each day by journaling 3 things youre grat...,0,0,0,0
3,first this shit and now jaguar pulling som bul...,1,1,1,1
4,and fuck you to the guy i comforted when anoth...,1,1,1,1
5,heir character design is horrendous sweetbabyi...,1,0,1,1
6,with the right people there is hope for change...,0,0,0,0
7,even if kd was playing the suns are just so sm...,1,1,1,1
8,trump was elected a mere couple weeks ago and ...,1,1,1,1
9,believe in your ability to create a better fut...,0,0,0,0


store the predicted labels to the new dataset

In [48]:
df.to_excel("final_cleaned_predicted__data.xlsx", index=False)

From the above training with the zero-shot methodology on un-cleaned and cleaned data and by abserving the evaluation metrics , i observed that the model performed very well in both the cases;slightly more with the cleaned case. The accuracy and F1 scores are very good even though the model is not finetuned based on my data;this show the model is pre-trained and fine-tuned very well intially. so , fine-tuning can make this almost perfect.

Thus,i conclude my project.

## THANK YOU